In [8]:
from models import scalers, decompositions, estimators, create_params_grid
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import classification_report_imbalanced
from imblearn.pipeline import Pipeline
import pandas as pd
import neptune.new as neptune
import os

In [2]:
run = neptune.init(project='projektmlcdv/ml-project-MW-MM', api_token = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwYmI4NjVmYS0zM2JlLTRkZTQtOTI5YS0zNjY4NzhjNGQ4YjYifQ==')

https://app.neptune.ai/projektmlcdv/ml-project-MW-MM/e/MLPROJ-18


In [3]:
scalers = { key: scalers[key] for key in ['StandardScaler']}
estimators = { key: estimators[key] for key in ['LogisticRegression']}#, 'DecisionTreeClassifier', 'SVC', 'RandomForestClassifier'] }
decompositions ={ key: decompositions[key] for key in ['kpca']}
params_grid = create_params_grid(scalers, decompositions, estimators)

#### Creating Pipeline

In [4]:
pipe = Pipeline(steps=[('scaler', StandardScaler()),('decomposition',PCA()),('estimator', LogisticRegression())])

#### GridSearch

In [5]:
grid = GridSearchCV(pipe, params_grid,verbose = 2, cv=3, n_jobs = -1, scoring = ['f1_weighted','recall_weighted','precision_weighted','roc_auc_ovo_weighted'],refit = False,)

In [6]:
encoder = lambda x:1 if x == -1 else 0
decoder = lambda x:-1 if x == 1 else 1

In [9]:
X = pd.read_csv(os.path.join('data','train_data.csv'), header=None)
y = pd.read_csv(os.path.join('data','train_labels.csv'), header=None, names=['y'])

In [10]:
y= y['y'].apply(encoder)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [12]:
best = grid.fit(X_train,y_train)

Fitting 3 folds for each of 4500 candidates, totalling 13500 fits


C:\Users\Marcin\anaconda3\envs\ml-mw-mm\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.85131356 ...        nan        nan        nan]
  warnings.warn(
C:\Users\Marcin\anaconda3\envs\ml-mw-mm\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.89722716 ...        nan        nan        nan]
  warnings.warn(
C:\Users\Marcin\anaconda3\envs\ml-mw-mm\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.80987117 ...        nan        nan        nan]
  warnings.warn(
C:\Users\Marcin\anaconda3\envs\ml-mw-mm\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.48811778 ...        nan        nan        nan]
  warnings.warn(


In [13]:
run["cv_results"].upload(neptune.types.File.as_html(pd.DataFrame.from_dict(best.cv_results_)))
run['all_params'] = npt_utils.get_estimator_params(best)

NameError: name 'npt_utils' is not defined